# Amazon Bedrock AgentCore Gateway - セマンティック検索チュートリアル

### チュートリアル詳細


| 情報                 | 詳細                                                                             |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ  | 対話型                                                                           |
| エージェントタイプ    | シングル                                                                         |
| AgentCore サービス   | AgentCore Gateway、AgentCore Identity                                            |
| エージェントフレームワーク | Strands Agents                                                               |
| LLM モデル           | Anthropic Claude Haiku 4.5                                                       |
| チュートリアル構成    | Lambda バックエンドの AgentCore Gateway を Strands Agent から作成・使用           |
| チュートリアル分野    | クロスバーティカル                                                                |
| 例の複雑さ           | 簡単                                                                             |
| 使用 SDK             | Amazon BedrockAgentCore Python SDK と boto3                                       |

### チュートリアルアーキテクチャ
Amazon Bedrock AgentCore Gateway は、エージェントとそれらが対話する必要があるツールやリソース間の統合された接続性を提供します。Gateway はこの接続層で複数の役割を果たします：

1. **セキュリティガード**：Gateway は OAuth 認可を管理し、有効なユーザー/エージェントのみがツール/リソースにアクセスできるようにします。
2. **トランスレーター**：Gateway は Model Context Protocol (MCP) などの一般的なプロトコルを使用して行われたエージェントリクエストを API リクエストと Lambda 呼び出しに変換します。これにより、開発者はサーバーのホスティング、プロトコル統合の管理、バージョンサポート、バージョンパッチングなどを管理する必要がなくなります。
3. **コンポーザー**：Gateway により、開発者は複数の API、関数、ツールを単一の MCP エンドポイントにシームレスに組み合わせて、エージェントが使用できるようになります。
4. **キーチェーン**：Gateway は適切なツールで使用する適切な認証情報の注入を処理し、エージェントが異なる認証情報セットを必要とするツールをシームレスに活用できるようにします。
5. **リサーチャー**：Gateway により、エージェントはすべてのツールを検索して、特定のコンテキストや質問に最適なものだけを見つけることができます。これにより、エージェントは一握りではなく数千のツールを活用できます。また、エージェントの LLM プロンプトで提供する必要があるツールセットを最小化し、レイテンシとコストを削減します。
6. **インフラストラクチャマネージャー**：Gateway は完全にサーバーレスで、組み込みの可観測性と監査機能を備えており、開発者がエージェントとツールを統合するための追加インフラストラクチャを管理する必要がありません。

![動作の仕組み](images/gw-arch-overview.png)

### チュートリアルの主な機能

* AWS Lambda バックエンドターゲットを持つ Amazon Bedrock AgentCore Gateway の作成
* AgentCore Gateway セマンティック検索の使用
* Strands Agents を使用して AgentCore Gateway 検索がレイテンシを改善する方法を示す

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents

## AgentCore Gateway は大量のツールを持つ MCP サーバーの課題解決を支援します
一般的なエンタープライズ環境では、エージェントビルダーは数百または数千の MCP ツールを持つ MCP サーバーに遭遇します。このツールの量は AI エージェントにとって課題となり、ツール選択精度の低下、コストの増加、過剰なツールメタデータによるトークン使用量増加に起因するレイテンシの増加が含まれます。これはエージェントをサードパーティサービス（例：Zendesk、Salesforce、Slack、JIRA など）や既存のエンタープライズ REST サービスに接続する際に発生する可能性があります。AgentCore Gateway はツール全体にわたる組み込みのセマンティック検索を提供し、エージェントのレイテンシ、コスト、精度を改善しながら、必要なツールをエージェントに提供します。ユースケース、LLM モデル、エージェントフレームワークによっては、一般的な MCP サーバーから数百のフルセットのツールを提供する場合と比較して、関連するツールにエージェントを集中させることで最大3倍のレイテンシ改善が見られます。

![動作の仕組み](images/gateway_tool_search.png)

## このノートブックで学ぶこと
このノートブックでは、AgentCore Gateway 検索のチュートリアルを提供します。このステップバイステップのチュートリアルを終えると、以下を理解できます：

- AgentCore Gateway の組み込み検索ツールを使用して関連するツールを素早く見つける方法
- ツール検索結果を Strands Agents に統合して、レイテンシを改善しコストを削減する方法

## ノートブック構造の概要
ノートブックは以下のセクションで構成されています：

1. AgentCore Gateway 検索の基礎を理解する
2. ノートブック環境の準備
3. 数百のツールを持つ Gateway のセットアップ
4. Gateway からのツール検索
5. 多数のツールを持つ MCP サーバーで Strands Agents を使用する
6. Strands Agent にツール検索結果を追加する
7. ツール検索を使用した3倍のレイテンシ改善を示す

# AgentCore Gateway 検索の基礎を理解する

AgentCore Gateway を作成する際、検索を有効にするかどうかを指定するオプションがあります。検索が有効な Gateway では、3つのことが起こります：

1. **ベクトルストアが作成される**。Gateway サービスは新しい Gateway 用にサーバーレスでフルマネージドなベクトルストアを自動的に作成します。これにより、Gateway ツール全体にわたる完全なセマンティック検索が可能になります。
2. **ベクトルストアにデータが投入される**。Gateway に Gateway ターゲットを追加すると、サービスは新しいターゲットからのツールに基づいてベクトルストアに自動的にエンベディングを使用してデータを投入します。ツールメタデータはツールの JSON 定義または REST サービスターゲットの OpenAPI Schema 仕様から取得されます。
3. **検索ツール（MCP ベース）が提供される**。すべてのユーザー定義ツール（AWS Lambda ターゲットまたは REST サービスから）に加えて、Gateway はセマンティック検索を提供する追加の MCP ツールを取得します。これは `x-amz-bedrock-agentcore-search` という名前です。プレフィックスにより、ユーザー定義ツールとの名前の衝突がないことが保証されます。将来的にはこのようなツールをさらに追加する可能性があります。検索ツールには `query` という単一の引数があります。検索ツールが呼び出されると、Gateway サービスはそのクエリを使用してセマンティック検索を実行し、利用可能なツールメタデータ（名前、説明、入出力スキーマ）に対してマッチングを行い、関連性の降順で最も関連性の高いツールを返します。

# ノートブック環境の準備

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

必要な Python ライブラリをすべてインポートし、環境変数をロードします

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.handlers import null_callback_handler

from strands.tools.mcp.mcp_client import MCPClient, MCPAgentTool

from mcp.client.streamable_http import streamablehttp_client
from mcp.types import Tool as MCPTool

import logging
import time
import json
import boto3
import requests
import utils

GATEWAY_NAME = "gateway-search-tutorial"

ロガーを設定します

In [ ]:
# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.ERROR)  # INFO) #DEBUG) #

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()]
)

boto3 のバージョンを確認します

In [ ]:
boto3.__version__

AgentCore コントロールプレーン API 用の boto3 クライアントを取得します。

In [ ]:
session = boto3.Session()
agentcore_client = session.client(
    "bedrock-agentcore-control",
)

# 数百のツールを持つ Gateway のセットアップ

AgentCore Gateway は、既存の API を MCP ツールとしてエージェントに公開するための安全でスケーラブルな方法を提供します。本番環境では、Gateway リソースは CloudFormation、CDK、Terraform などのツールを使用した Infrastructure as Code で作成されます。このチュートリアルでは、boto3 コントロールプレーン API を直接使用して、リソースと API をより効果的に理解できるようにします。これにより、独自の Gateway の構築と使用、より強力で安全なエージェントの作成をより簡単に始めることができます。

Gateway のセットアップの高レベルな手順は以下の通りです：

1. インバウンド（エージェントが Gateway を呼び出す）とアウトバウンド（Gateway がツールを呼び出す）のセキュリティに使用する ID プロバイダーと認証情報プロバイダーを定義する。
2. `create_gateway` を使用して Gateway を作成する。
3. `create_gateway_target` を使用して Gateway ターゲットを追加し、AWS Lambda または既存の RESTful サービスで実装される MCP ツールを公開する。

このチュートリアルでは、ID プロバイダー（IdP）として Amazon Cognito を、ターゲットとして AWS Lambda 関数を、アウトバウンド認証に AWS IAM を使用します。このチュートリアルで示される概念は、他の IdP や他のターゲットタイプを使用する場合にも同様に適用されます。

### Amazon Cognito リソースの作成

このチュートリアルでは、以下のリソースが既に作成され、対応する環境変数が設定されていることを前提としています：

- AWS Lambda 実行用の IAM ロール（`gateway_lambda_iam_role`）
- シンプルな数学ツール用の AWS Lambda 関数（`calc_lambda_arn`）
- レストラン予約ツール用の AWS Lambda 関数（`restaurant_lambda_arn`）
- Amazon Cognito ユーザープール、クライアント ID（`cognito_client_id`）とディスカバリー URL（`cognito_discovery_url`）を提供

レストラン API の JSON ツールメタデータを見てみましょう。既存の REST サービスと統合する場合は、代わりに OpenAPI Schema を使用して API 仕様が提供されることに注意してください。

In [ ]:
with open("./restaurant/restaurant-api.json") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

こちらはシンプルな電卓 API です。

In [ ]:
with open("./calc/calc-api.json") as f:
    data = json.load(f)[0:3]
print(json.dumps(data, indent=4))

こちらは電卓ツールの AWS Lambda 関数実装です。

In [ ]:
from IPython.display import display, Code

with open("./calc/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
with open("./restaurant/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
#### MCP ツールに変換するサンプル AWS Lambda 関数を作成
calc_lambda_resp = utils.create_gateway_lambda(
    "calc/lambda_function_code.zip", lambda_function_name="calc_lambda_gateway"
)

if calc_lambda_resp is not None:
    if calc_lambda_resp["exit_code"] == 0:
        print(
            "Lambda 関数を作成しました、ARN: ",
            calc_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda 関数の作成に失敗しました、メッセージ: ",
            calc_lambda_resp["lambda_function_arn"],
        )

In [ ]:
calc_lambda_resp["lambda_function_arn"]

In [ ]:
#### MCP ツールに変換するサンプル AWS Lambda 関数を作成
restaurant_lambda_resp = utils.create_gateway_lambda(
    "restaurant/lambda_function_code.zip",
    lambda_function_name="restaurant_lambda_gateway",
)

if restaurant_lambda_resp is not None:
    if restaurant_lambda_resp["exit_code"] == 0:
        print(
            "Lambda 関数を作成しました、ARN: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda 関数の作成に失敗しました、メッセージ: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )

In [ ]:
restaurant_lambda_resp["lambda_function_arn"]

In [ ]:
cognito_response = utils.setup_cognito_user_pool()

In [ ]:
bearer_token = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)

In [ ]:
gateway_role_arn = utils.create_gateway_iam_role(
    lambda_arns=[
        calc_lambda_resp["lambda_function_arn"],
        restaurant_lambda_resp["lambda_function_arn"],
    ]
)

#### コントロールプレーン API を使用するためのヘルパー関数を作成しましょう

In [ ]:
def read_apispec(json_file_path):
    try:
        # read json file and return contents as string
        with open(json_file_path, "r") as file:
            # Parse JSON to Python object
            api_spec = json.load(file)
            return api_spec

    except FileNotFoundError:
        return f"Error: File {json_file_path} not found"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"


def list_gateways():
    response = agentcore_client.list_gateways()
    print(json.dumps(response, indent=2, default=str))
    return response

#### Gateway 作成ヘルパー関数
こちらは名前と説明を指定して AgentCore Gateway を作成するためのヘルパー関数です。IdP として Amazon Cognito を使用し、許可されたクライアント ID とディスカバリー URL を環境変数から取得します（既に定義されています）。また、デフォルトで結果の Gateway でセマンティック検索を有効にし、事前定義された IAM ロールを使用します。

In [ ]:
def create_gateway(gateway_name, gateway_desc):
    # Use Cognito for Inbound OAuth to our Gateway
    auth_config = {
        "customJWTAuthorizer": {
            "allowedClients": [cognito_response["client_id"]],
            "discoveryUrl": cognito_response["discovery_url"],
        }
    }
    # Enable semantic search of tools
    search_config = {
        "mcp": {"searchType": "SEMANTIC", "supportedVersions": ["2025-03-26"]}
    }
    # Create the gateway
    response = agentcore_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        authorizerType="CUSTOM_JWT",
        description=gateway_desc,
        protocolType="MCP",
        authorizerConfiguration=auth_config,
        protocolConfiguration=search_config,
    )
    # print(json.dumps(response, indent=2, default=str))
    return response["gatewayId"]

#### Gateway ターゲット作成ヘルパー関数
この関数は既存の Gateway に新しい AWS Lambda ターゲットを作成します。Gateway ID、新しいターゲットの名前と説明、既存の AWS Lambda 関数の ARN、Gateway から公開するツールのインターフェースを記述する JSON スキーマを提供するだけです。

In [ ]:
def create_gatewaytarget(gateway_id, target_name, target_descr, lambda_arn, api_spec):
    # Add a Lambda target to the gateway
    response = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        description=target_descr,
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_arn,
                    "toolSchema": {"inlinePayload": api_spec},
                }
            }
        },
        # Use IAM as credential provider
        credentialProviderConfigurations=[
            {"credentialProviderType": "GATEWAY_IAM_ROLE"}
        ],
    )
    return response["targetId"]

### 最初の AgentCore Gateway の作成
最初の Gateway をセットアップする前に、Gateway がインバウンドリクエスト（MCP ツールを使用するため）とアウトバウンドアクセス（Gateway からツールとリソースへ）の両方に対してどのようにセキュリティを提供するかを見てみましょう。

![動作の仕組み](images/gateway_secure_access.png)

それでは、名前と説明を指定してこのチュートリアル用の Gateway を作成しましょう。

In [ ]:
print(f"ゲートウェイを作成中、名前: {GATEWAY_NAME}")
gatewayId = create_gateway(
    gateway_name=GATEWAY_NAME, gateway_desc="AgentCore Gateway Tutorial"
)
print(f"ゲートウェイを作成しました、ID: {gatewayId}.")

### AgentCore Gateway ターゲットの追加
このチュートリアルでは、Lambda 関数のペアが既にインストールされていると仮定します。1つはシンプルな数学計算用、もう1つはレストラン予約のシミュレーション用です。これらの関数それぞれに Gateway ターゲットを追加します。

これらのターゲットを追加したら、AgentCore Gateway 検索の威力を示すために、より多くの MCP ツール数を増やすための追加ターゲットを追加します。

Gateway が作成されたので、Lambda 関数を通じてレストラン予約を行うためのターゲットを追加しましょう。

In [ ]:
restaurant_api_spec = read_apispec("./restaurant/restaurant-api.json")
restaurant_lambda_arn = restaurant_lambda_resp["lambda_function_arn"]
print(f"レストラン Lambda ARN: {restaurant_lambda_arn}")

restaurantTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=restaurant_lambda_arn,
    target_name="FoodTools",
    target_descr="Restaurant Tools",
    api_spec=restaurant_api_spec,
)
print(f"RestaurantTarget を作成しました、ID: {restaurantTargetId}、Gateway: {gatewayId}")

ここでは2番目のターゲットを追加します。今回は4つの基本ツール（加算、減算、乗算、除算）を実装する Lambda と、投資管理（トレーディング、クレジットリサーチ、定量分析、ポートフォリオ管理）用の75個の生成されたツール定義のセットを使用します。投資管理ツール定義は実際には Lambda 関数に実装されていません。大量のツールを示すためにのみ追加しています。

In [ ]:
calc_api_spec = read_apispec("./calc/calc-api.json")
print(f"計算用 API スペックの関数数: {len(calc_api_spec)}\n")
calc_lambda_arn = calc_lambda_resp["lambda_function_arn"]
print(f"計算 Lambda ARN: {calc_lambda_arn}")

time.sleep(5)
calcTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=calc_lambda_arn,
    target_name="CalcTools",
    target_descr="Calculation Tools",
    api_spec=calc_api_spec,
)
print(f"CalcTools ターゲットを作成しました、ID: {calcTargetId}、Gateway: {gatewayId}")

Gateway 検索の威力を示すために、Calculator ターゲットのコピーをいくつか追加して、最終的に300以上の MCP ツールが公開されるようにします。

In [ ]:
def add_more_tools(gatewayId):
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc2",
        target_descr="Calculation 2 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc2 ターゲットを作成しました、ID: {calcTargetId}、Gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc3",
        target_descr="Calculation 3 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc3 ターゲットを作成しました、ID: {calcTargetId}、Gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc4",
        target_descr="Calculation 4 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc4 ターゲットを作成しました、ID: {calcTargetId}、Gateway: {gatewayId}")

In [ ]:
add_more_tools(gatewayId=gatewayId)

In [ ]:
resp = agentcore_client.list_gateway_targets(gatewayIdentifier=gatewayId)
targets = resp["items"]
for target in resp["items"]:
    print(f"{target['name']} - {target['description']}")

# Gateway からのツール検索

### 検索前に MCP ツールリストに慣れる

指定された Gateway ID の MCP エンドポイント URL を取得するユーティリティ関数と、Gateway を安全に使用するための JWT OAuth アクセストークンを取得するユーティリティ関数を定義しましょう。

In [ ]:
def get_gateway_endpoint(gateway_id):
    response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = response["gatewayUrl"]
    return gateway_url

Gateway が作成されターゲットが追加されたので、その Gateway への MCP URL を取得しましょう。Gateway ID に基づいて Gateway コントロールプレーンからエンドポイント URL を取得できます。

#### Gateway に対して MCP Inspector を使用する

MCP サーバーのエンドポイント URL と JWT ベアラートークンがあるので、MCP Inspector ツールで MCP サーバーを探索したいかもしれません。MCP Inspector は任意の MCP サーバーに接続でき、提供されるツールをリストし、使いやすいツール呼び出し体験を提供するオープンソースツールです。

ターミナルウィンドウから `npx @modelcontextprotocol/inspector` と入力するだけで MCP Inspector を起動できます。次に、Gateway エンドポイント URL と JWT トークンを貼り付けて接続します。接続したら、List Tools と Invoke Tool を試してみてください。

サンプルのスクリーンショットを示します。

![MCP Inspector](images/mcp_inspector.png)

In [ ]:
gatewayEndpoint = get_gateway_endpoint(gateway_id=gatewayId)
print(f"ゲートウェイエンドポイント - MCP URL: {gatewayEndpoint}")

MCP サーバーのセキュリティは OAuth に基づいています。Gateway と対話するには、IdP から JWT OAuth アクセストークンを取得する必要があります。

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
print(f"Bearer トークン: {jwtToken}")

In [ ]:
# !npx @modelcontextprotocol/inspector

#### jsonrpc を使用して MCP ツールを呼び出したりリストしたりするヘルパー関数を作成
`invoke_gateway_tool` というヘルパー関数を定義しましょう。jsonrpc を使用して MCP サーバー（もちろん Gateway を含む）が公開する任意の MCP ツールを呼び出します。エンドポイント URL と JWT トークンがあれば、このユーティリティを使用して、Gateway に Gateway ターゲットを追加した際に AgentCore Gateway が利用可能にした任意の MCP ツールを呼び出すことができます。

In [ ]:
def invoke_gateway_tool(gateway_endpoint, jwt_token, tool_params):
    # print(f"ツールを呼び出し中 {tool_params['name']}")

    requestBody = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/call",
        "params": tool_params,
    }
    response = requests.post(
        gateway_endpoint,
        json=requestBody,
        headers={
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        },
    )

    return response.json()

こちらは MCP の `tools/list` メソッドを使用して Gateway から利用可能な MCP ツールをリストするための別のユーティリティ関数です。Gateway ID と JWT トークンを指定すると、その Gateway からフルセットのツールを取得し、エージェントで使用可能な形式のリストを返します。返されるリストには、エージェントに渡すのに適した Strands Agents の MCPAgentTool オブジェクトが含まれています。

`tools/list` 呼び出しはページネーションされているため、関数は `nextCursor` フィールドが設定されなくなるまで、一度に1ページずつツールを取得しながらループする必要があります。このユーティリティ関数は HTTPS と jsonrpc プロトコルを使用してエンドポイントを直接呼び出します。これは Strands Agents が提供する `MCPClient` クラスと比較して、より低レベルなツールリスト取得方法です。その体験は後で見ていきます。

In [ ]:
def get_all_agent_tools_from_mcp_endpoint(gateway_endpoint, jwt_token, client):
    more_tools = True
    tools_count = 0
    tools_list = []

    requestBody = {"jsonrpc": "2.0", "id": 2, "method": "tools/list", "params": {}}
    next_cursor = ""

    while more_tools:
        if tools_count == 0:
            requestBody["params"] = {}
        else:
            print(f"\n次のカーソルが返されたため、次のページのツールを取得中\n")
            requestBody["params"] = {"cursor": next_cursor}

        headers = {
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        }

        print(f"\n\nGateway {gateway_endpoint} のツールをリスト中")

        response = requests.post(gateway_endpoint, json=requestBody, headers=headers)

        tools_json = response.json()
        tools_count += len(tools_json["result"]["tools"])

        for tool in tools_json["result"]["tools"]:
            mcp_tool = MCPTool(
                name=tool["name"],
                description=tool["description"],
                inputSchema=tool["inputSchema"],
            )
            mcp_agent_tool = MCPAgentTool(mcp_tool, client)
            short_descr = tool["description"][0:40] + "..."
            print(f"ツールを追加中 '{mcp_agent_tool.tool_name}' - {short_descr}")
            tools_list.append(mcp_agent_tool)

        if "nextCursor" in tools_json["result"]:
            next_cursor = tools_json["result"]["nextCursor"]
            more_tools = True
        else:
            more_tools = False

    print(f"\n見つかったツールの合計: {tools_count}\n")
    return tools_list

このヘルパー関数を使用して結果を確認しましょう。

In [ ]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_agent_tools_from_mcp_endpoint(
        gateway_endpoint=gatewayEndpoint, jwt_token=jwtToken, client=client
    )
    print(f"\njsonrpc を使用した MCP ツールリストで {len(all_tools)} 件のツールが見つかりました\n")

#### ページネーション付きの Strands Agents list_tools_sync() を使用する
Python ベースの MCP クライアントを書いたことがある場合、クライアントが関連付けられている MCP サーバーから利用可能なツールセットを返す `list_tools_sync()` メソッドをご存知でしょう。しかし、MCP のツールリストもページネーションされていることをご存知でしたか？デフォルトでは、最初の小さなツールのサブセットのみが返されます。シンプルな MCP サーバーでは気づかなかったかもしれませんが、多くの実際の MCP サーバーでは、コードはページが残っていなくなるまで一度に1ページずつツールを取得しながらループする必要があります。以下のユーティリティ `get_all_mcp_tools_from_mcp_client` はまさにそれを行います。指定された Strands Agent MCP クライアントからフルセットのツールを返します。

In [ ]:
def get_all_mcp_tools_from_mcp_client(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools

Gateway で試して、Python クライアントが見つけるツール数を確認しましょう。
まず、エンドポイント URL と JWT ベアラートークンに基づいて MCPClient オブジェクトを作成します。次に、MCP サーバーから返される多くのページにわたるフルセットのツールを取得します。
先ほど追加したターゲットを考慮すると、300以上のツールが返されるはずです。

In [ ]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nmcp クライアントの list_tools_sync() で {len(all_tools)} 件のツールが見つかりました\n")

これで、Gateway を MCP サーバーとして使用してフルセットのツールを取得する3つの異なる方法を見てきました：

1. jsonrpc を直接使用
2. Strands Agent MCPClient の `list_tools_sync()` メソッドを使用
3. MCP Inspector ツールを使用（内部的に jsonrpc を使用）

エージェントを構築する一般的な開発者にとっては、オプション2を使用することになります。

### 組み込みの Gateway セマンティック検索ツールを使用する
それでは、追加の MCP ツールとして提供される組み込みの検索ツールを使用して、Gateway で最初のセマンティック検索を試してみましょう。

まず、MCP を使用して検索ツールを実行するためのシンプルなユーティリティ関数を定義しましょう。ツールのリストと同様に、Gateway エンドポイントと JWT トークンが必要です。それ以外に必要なのは検索クエリを渡すだけです。Gateway 検索ツールは残りの処理を行い、そのクエリを自動的に管理するサーバーレスベクトルストアとマッチングします。

In [ ]:
def tool_search(gateway_endpoint, jwt_token, query):
    toolParams = {
        "name": "x_amz_bedrock_agentcore_search",
        "arguments": {"query": query},
    }
    toolResp = invoke_gateway_tool(
        gateway_endpoint=gateway_endpoint, jwt_token=jwt_token, tool_params=toolParams
    )
    tools = toolResp["result"]["structuredContent"]["tools"]
    return tools

In [ ]:
start_time = time.time()
tools_found = tool_search(
    gateway_endpoint=gatewayEndpoint,
    jwt_token=jwtToken,
    query="find me 3 credit research tools",
)
end_time = time.time()
print(
    f"Gateway 直接呼び出しによるツール検索は {(end_time - start_time):.2f} 秒かかりました"
)
print(f"トップツール: {tools_found[0]['name']}")

検索が1秒未満で返されることに注目してください。結果はクエリとツールメタデータのマッチングに基づいて検索関連性の降順で返されます。最も関連性の高いツールがリストの先頭にあります。検索の初期実装では最大10件の結果が返されます。その後、これらすべてのツールをエージェントで使用するか、最も関連性の高いマッチのサブセットのみを選択できます。

# 多数のツールを持つ MCP サーバーで Strands Agents を使用する

まず、Strands Agent で使用するモデルを選択します。このノートブックでは Amazon Bedrock モデルを使用していますが、Strands と AgentCore は任意の LLM で動作できます。

In [ ]:
bedrockmodel = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    temperature=0.7,
    streaming=True,
    boto_session=session,
)

#### AgentCore Gateway をエージェントツールに使用するシンプルな Strands Agent
Strands Agent を使用して AgentCore Gateway が提供する MCP サーバーを活用することがいかに簡単かを示しましょう。このシンプルな例では、エージェントに数字を加算するよう依頼します。

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nmcp クライアントの list_tools_sync() で {len(all_tools)} 件のツールが見つかりました\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    result = simple_agent("100 + 50 を計算して")
    print(f"{result.message['content'][0]['text']}")

Strands Agents フレームワークでは、エージェントのイベントループをバイパスして MCP ツールを直接呼び出すこともできます。
Gateway ツールはネイティブ MCP ツールとして公開されているため、Gateway ツールに対してもこれを行うことができます。ここでは `agent.tool.<tool_name>(args)` 構文を使用して Gateway MCP ツールを呼び出します：

```python
direct_result = simple_agent.tool.Calc2___add_numbers(firstNumber=10, secondNumber=20)
resp_json = json.loads(direct_result['content'][0]['text'])
```

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nmcp クライアントの list_tools_sync() で {len(all_tools)} 件のツールが見つかりました\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    direct_result = simple_agent.tool.Calc2___add_numbers(
        firstNumber=10, secondNumber=20
    )
    print(f"直接結果 = {direct_result}")

In [ ]:
def get_search_tool(client):
    mcp_tool = MCPTool(
        name="x_amz_bedrock_agentcore_search",
        description="A special tool that returns a trimmed down list of tools given a context. Use this tool only when there are many tools available and you want to get a subset that matches the provided context.",
        inputSchema={
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "search query to use for finding tools",
                }
            },
            "required": ["query"],
        },
    )
    return MCPAgentTool(mcp_tool, client)

In [ ]:
def search_using_strands(client, query):
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(query=query)

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    # print(json.dumps(search_results, indent=4))
    return search_results

In [ ]:
def find_strands_tools(client, query, top_n):
    strands_mcp_tools = []
    results = search_using_strands(client, query)
    for tool in results[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(
        query="find equity trading tools"
    )

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    print(json.dumps(search_results, indent=4))

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    results = search_using_strands(client, "find trading tools")
    print(json.dumps(search_results[0], indent=4))

    results = search_using_strands(client, "find credit research tools")
    print(json.dumps(search_results[0], indent=4))

# Strands Agent にツール検索結果を追加する

次に、検索から返されたツールを Strands Agent に追加する方法を見てみましょう。コーディングを簡単にするために、ツール検索結果を Strands MCPAgentTool オブジェクトにマッピングするユーティリティ関数を提供しましょう。検索結果を渡し、それらの結果のうちいくつをエージェントに渡すかを指定するだけです。

In [ ]:
def tools_to_strands_mcp_tools(tools, top_n):
    strands_mcp_tools = []
    for tool in tools[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    agent = Agent(
        model=bedrockmodel,
        tools=find_strands_tools(
            client,
            "tools for doing addition, subtraction, multiplication, division",
            10,
        ),
    )
    result = agent("(10*2)/(5-3)")
    print(f"{result.message['content'][0]['text']}")

In [ ]:
%%time

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    print("フルセットのツールを持つエンドポイントから加算ツールを検索中...")
    tools_found = tool_search(
        gateway_endpoint=gatewayEndpoint,
        jwt_token=jwtToken,
        query="tools for multiplying two numbers",
    )
    print(f"トップツール: {tools_found[0]['name']}\n")

    agent = Agent(model=bedrockmodel, tools=tools_to_strands_mcp_tools(tools_found, 1))
    result = agent("10 * 70 を計算して")
    print(f"{result.message['content'][0]['text']}")

レイテンシの改善に注目してください。Gateway 検索からのツールのサブセットを使用するこの例は、数百のツールに依存する場合のエージェント呼び出しよりも大幅に高速です。

# ツール検索を使用した3倍のレイテンシ改善を示す

Strands エージェントから Gateway MCP ツールを使用する方法と、ツールを検索してエージェントに追加する方法がわかったので、検索の威力を示しましょう。大幅なレイテンシ削減と入力トークン使用量の削減を強調します。

レイテンシとトークン削減を示すために、2つのアプローチを並べて比較します：

1. **検索なし**。MCP サーバーが公開するフルセットの MCP ツール（この場合300以上）をエージェントに追加し、エージェントにツール選択と呼び出しを任せます。
2. **検索を使用**。2番目のアプローチでは、手元のトピックに基づいて検索を行い、最も関連性の高いツールのみをエージェントに送信します。ポイントを証明するために、数学（数字の加算）と食べ物（レストラン予約の予約）の2つの異なるトピックを使用し、それぞれ異なるツールセットが必要です。

レイテンシ分布を正規化し、意味のある比較を得るために、各アプローチで複数回の反復を実行します。また、利点を誇張しないために、検索アプローチを行う際には、エージェント呼び出しのレイテンシだけでなく、ツール検索を実行するレイテンシも含めます。各反復では、エージェントに2つのタスクを渡します：

1. 数学タスク - 2つの数字を加算
2. 食べ物タスク - レストラン予約を行う

以下の結果は利点を示し、3倍のレイテンシ削減と、入力トークン使用量のさらに大きな削減を強調しています。トークン使用量の節約はコスト削減につながりますが、入力トークンの比較的低いコスト（多くのモデルプロバイダーでは入力トークンははるかに安価）のため、それほど大きな影響がない場合があります。それでも、大規模なエージェント展開では、入力トークン使用コストも積み重なる可能性があるため、動的検索はエージェントランタイムコストの削減にも役立ちます。

#### MCP ツールのフルセットを使用するエージェントのレイテンシとトークン使用量を測定

In [ ]:
iterations = 2
full_tokens = light_tokens = 0
full_elapsed_time = light_elapsed_time = 0

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)

In [ ]:
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nmcp クライアントの list_tools_sync() で {len(all_tools)} 件のツールが見つかりました\n")
    heavy_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )

    math_input = "<iteration> を 100 に加算して"
    food_input = (
        "8月<day>日午後7時に Jo Smith の名前で2名用のテーブルをバーガーキングで予約して"
    )

    print("すべてのツールでエージェントを使用中...")
    start_time = time.time()

    for i in range(iterations):
        result = heavy_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

        result = heavy_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

    end_time = time.time()
    full_tokens = result.metrics.accumulated_usage["totalTokens"]
    full_elapsed_time = end_time - start_time
    print(f"\n合計時間: {full_elapsed_time:.1f} 秒、トークン数: {full_tokens:,d}\n")

#### Gateway 検索を使用するエージェントのレイテンシとトークン使用量を測定
次に、検索を呼び出して関連するツールを見つけ、それらの関連するツールのみでエージェントを呼び出す動的アプローチを使用します。各会話ターンでエージェントをリセットしているため、前のターンの会話履歴からメッセージリストも初期化します。

In [ ]:
with client:
    print("フォーカスされた検索からのツールのみでエージェントを使用中...")
    start_time = time.time()
    messages = []

    light_agent = Agent()

    for i in range(iterations):
        print("フルセットのツールを持つエンドポイントから加算ツールを検索中...")
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for simply adding two numbers",
        )
        print(f"トップツール: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages

        print(
            "フルセットのツールを持つエンドポイントからレストラン予約ツールを検索中..."
        )
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for booking a restaurant reservation",
        )
        print(f"トップツール: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages
        light_tokens = light_result.metrics.accumulated_usage["totalTokens"]
    end_time = time.time()

    light_elapsed_time = end_time - start_time
    print(f"\n合計時間: {light_elapsed_time:.1f} 秒、トークン数: {light_tokens:,d}\n")

#### 結果を比較し、検索の利点を強調

In [ ]:
print(
    f"\n\n検索なしのレイテンシ: {full_elapsed_time:.1f}秒、検索使用時: {light_elapsed_time:.1f}秒"
)
print(f"検索なしのトークン数: {full_tokens:,d}、検索使用時: {light_tokens:,d}")

# 結論
このチュートリアルでは、Amazon Bedrock AgentCore Gateway とその組み込みのフルマネージドセマンティック検索機能について学びました。以下のことを見てきました：

- セマンティック検索を有効にして Gateway を作成する方法
- 複数の Gateway ターゲットを追加して単一のエンドポイントから300以上の MCP ツールを公開する方法
- 3つの異なるアプローチを使用して Gateway のツールをリストする方法
- 組み込みのセマンティック検索ツールを使用して関連するツールを見つける方法
- Strands Agent と検索を統合する方法
- 数百のツールを持つサーバーを使用するエージェントと、セマンティック検索を使用して特定のトピックにツールを絞り込むエージェントのパフォーマンスを比較する方法

AgentCore Gateway 検索は、より高度なユースケースにも役立ちます。検索をコントロールプレーン API だけでなくネイティブ MCP ツールとして提供することで、エージェントにより多くの自律性を与えて新しい MCP サーバーを発見し、実行時に新しい機能を見つけることができ、より困難な問題を解決するための突破口につながります。さらに、検索は MCP レジストリの重要な基盤であり、エージェント開発者が新しいエージェントを設計・構築する際のサポートにもなります。

# リソースのクリーンアップ

まず、AgentCore Gateway リソースをクリーンアップするためのヘルパー関数を定義しましょう。

In [ ]:
def delete_gatewaytarget(gateway_id):
    response = agentcore_client.list_gateway_targets(gatewayIdentifier=gateway_id)

    print(f"Gateway に {len(response['items'])} 件のターゲットが見つかりました")

    for target in response["items"]:
        print(
            f"ターゲットを削除中、名前: {target['name']}、ID: {target['targetId']}"
        )

        response = agentcore_client.delete_gateway_target(
            gatewayIdentifier=gateway_id, targetId=target["targetId"]
        )
        time.sleep(20)


def delete_gateway(gateway_id):
    response = agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)

### Gateway ターゲットの削除

In [ ]:
delete_gatewaytarget(gateway_id=gatewayId)

### Gateway 自体の削除

In [ ]:
delete_gateway(gateway_id=gatewayId)

In [ ]:
lambda_arns = [
    calc_lambda_resp["lambda_function_arn"],
    restaurant_lambda_resp["lambda_function_arn"],
]

for arn in lambda_arns:
    if utils.delete_gateway_lambda(arn):
        print(f"Lambda を削除しました: {arn}")
    else:
        print(f"Lambda が見つからないか削除に失敗しました: {arn}")

In [ ]:
# Gateway ロールのクリーンアップ
if utils.delete_gateway_iam_role():
    print("Gateway IAM ロールを削除しました")
else:
    print("Gateway IAM ロールが見つからないか削除に失敗しました")

# Cognito クリーンアップ
if utils.delete_cognito_user_pool():
    print("Cognito プールを削除しました")
else:
    print("Cognito プールの削除に失敗しました")